In [1]:
# --- Import Libraries ---
import pandas as pd
import numpy as np
import re, warnings
from datetime import datetime


warnings.filterwarnings("ignore")

# --- Google Sheets libraries ---
import gspread
from google.oauth2.service_account import Credentials

In [2]:
#===================================================
# Load raw data
# ===================================================

customers = pd.read_csv(r"C:\Users\hp\Desktop\Dataset\dirty_customers.csv")
transactions = pd.read_csv(r"C:\Users\hp\Desktop\Dataset\dirty_transactions.csv")


In [4]:
# ===  Basic inspection ===
print("\n🔍 Initial Data Overview:")
print("="*50)

# 1 Tables Shape
print(f"Customers shape: {customers.shape}")
print(f"Transactions shape: {transactions.shape}")

# 2 Column data types
print("\n🔹 Column Data Types (Customers):")
print(customers.dtypes)
print("\n🔹 Column Data Types (Transactions):")
print(transactions.dtypes)

# 3 Missing values
print("\n🔹 Missing Values (Customers):")
print(customers.isna().sum())
print("\n🔹 Missing Values (Transactions):")
print(transactions.isna().sum())

# 4 Duplicate counts
print("\n🔹 Duplicate Counts:")
print(f"Customers duplicates: {customers.duplicated(subset=['customer_id']).sum()}")
print(f"Transactions duplicates: {transactions.duplicated(subset=['transaction_id']).sum()}")

# 5 Quick numeric stats
print("\n🔹 Numeric Summary (Customers):")
print(customers.describe())
print("\n🔹 Numeric Summary (Transactions):")
print(transactions.describe())



🔍 Initial Data Overview:
Customers shape: (1500, 15)
Transactions shape: (3000, 10)

🔹 Column Data Types (Customers):
customer_id          object
full_name            object
email                object
country              object
signup_date          object
subscription_plan    object
plan_price           object
payment_method       object
is_trial             object
renewal_date         object
churn_flag           object
total_logins         object
last_login_date      object
lifetime_value       object
customer_feedback    object
dtype: object

🔹 Column Data Types (Transactions):
transaction_id       object
customer_id          object
transaction_date     object
amount_paid          object
payment_status       object
payment_channel      object
refund_flag          object
invoice_number       object
discount_code        object
support_ticket_id    object
dtype: object

🔹 Missing Values (Customers):
customer_id           16
full_name              0
email                  0
country   

In [5]:
print("\n🔍 Initial Data Preview")
print("="*50)

print(customers.head())
print(transactions.head())



🔍 Initial Data Preview
                            customer_id         full_name  \
0  1a3d1fa7-bc89-40a9-a3b8-c1e9392456de      allison hill   
1  96da1dac-72ff-4d2a-b86e-cbe06b65a6a4    michelle miles   
2  89463e85-759c-4e66-bacf-b3d00b1f9163   Gregory baker     
3  29a3b2e9-5d65-4441-9588-42dea2bc372f     Brian Ramirez   
4  c6a7ee39-c4b0-42cc-97c5-24a55304317f  Melissa robinson   

                   email country signup_date subscription_plan plan_price  \
0      allison@gmail.com   India         NaN               NaN          0   
1    michelleoutlook.com     uk   2000-03-21              Pro          49   
2  gregory @ outlook.com  Germny  2021-01-22              Free         0    
3      brian @ gmail.com  Germny         NaN               pro         49   
4  melissa @ outlook.com     U.S  2023-13-01               NaN         49   

  payment_method is_trial renewal_date churn_flag total_logins  \
0    credit card      NaN   2024-08-21       TRUE     20 times   
1    Credit Ca

In [6]:
# ================================================
#  Store Raw Copies BEFORE Any Cleaning Happens
# ================================================
raw_customers = customers.copy()
raw_transactions = transactions.copy()


In [7]:
# ------------------------------
# Helper function for Clean date formats
# ------------------------------
def _clean_date_helper(date_str):
    """Handles multiple date formats and coerces invalid dates to NaT."""
    if pd.isna(date_str) or date_str in [None, 'nan', 'NULL']:
        return pd.NaT
    date_str = str(date_str).strip()
    
    # Handle "Jan 2023", "Feb 2024"
    if re.match(r'^[A-Za-z]{3}\s\d{4}$', date_str):
        return pd.to_datetime(date_str, format='%b %Y', errors='coerce')
    
    # Handle standard formats (e.g., YYYY-MM-DD, MM/DD/YYYY, etc.)
    return pd.to_datetime(date_str, errors='coerce')


In [8]:
# ------------------------------
# 🧼 1. Clean Customers Table
# ------------------------------
def clean_customers(df):
    print("\n🧼 Cleaning Customers Data...")

    
    ## Step 1: Standardize Column Names
    df.columns = (
        df.columns.str.lower()
        .str.replace(" ", "_")
        .str.replace("_-", "", regex=False)
    )

    
    ## Step 2: Date Column Cleaning
    for col in ["signup_date", "renewal_date", "last_login_date"]:
        if col in df.columns:
            df[col] = df[col].apply(_clean_date_helper)

    ## Step 3: Numeric Columns Cleaning
    # General cleanup for currency/numeric fields: plan_price, lifetime_value
    for col in ["plan_price", "lifetime_value"]:
        if col in df.columns:
            df[col] = (
                df[col].astype(str)
                .str.replace(r"[^\d.\-]", "", regex=True) # Remove $ and comma
            )
            df[col] = pd.to_numeric(df[col], errors="coerce")  # Convert to numeric
            
    
    # total_logins: remove " times" and convert to integer
    if "total_logins" in df.columns:
        df["total_logins"] = (
            df["total_logins"].astype(str)
            .str.replace(" times", "", regex=False)
        )
        df["total_logins"] = pd.to_numeric(df["total_logins"], errors="coerce")
        

    ## Step 4: Categorical Column Standardization
    country_map = {
        "Can": "Canada", "can ada": "Canada", "Canda": "Canada",
        "De": "Denmark", "Germny": "Germany",
        "india": "India", "INDIA": "India",
        "Nigerai": "Nigeria", "NIgeria": "Nigeria", "Naija": "Nigeria",
        "uk": "United Kingdom", "United Kngdom": "United Kingdom", "U.k": "United Kingdom",
        "U.S": "USA", "U.S.A": "USA", "us": "USA", "Usa": "USA"
    }

    if "country" in df.columns:
        df["country"] = df["country"].astype(str).str.strip().replace(country_map).str.title()

    df["full_name"] = df["full_name"].astype(str).str.title()
    df["email"] = df["email"].astype(str).str.strip().str.lower().str.replace(" ", "", regex=False)
    df["subscription_plan"] = df["subscription_plan"].astype(str).str.strip().str.title()

    
     ## Step 5: Boolean and Missing Value Cleaning
    boolean_map = {"TRUE": True, "YES": True, "Y": True, "1": True,
                   "FALSE": False, "NO": False, "N": False, "0": False}

    for col in ["is_trial", "churn_flag"]:
        if col in df.columns:
            df[col] = (
                df[col].astype(str)
                .str.strip().str.upper()
                .map(boolean_map)
                .astype("boolean")   # ensure proper boolean dtype
            )

    # Remove duplicate customers 
    duplicates_removed = df.duplicated(subset=["customer_id"]).sum()
    df.drop_duplicates(subset=["customer_id"], inplace=True)

    print(f"✅ Customers cleaned successfully! Removed {duplicates_removed} duplicates.")
    return df


# ------------------------------
# 🧽 2. Clean Transactions Table
# ------------------------------
def clean_transactions(df):
    print("\n🧽 Cleaning Transactions Data...")

    
    ## Step 1: Standardize Column Names
    df.columns = (
        df.columns.str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("_-", "_", regex=False)
    )

    ## Step 2: Date Column
    if "transaction_date" in df.columns:
        df["transaction_date"] = df["transaction_date"].apply(_clean_date_helper)
        
        
    ## Step 3:  Clean Amount Paid
    if "amount_paid" in df.columns:
        df["amount_paid"] = (
            df["amount_paid"].astype(str)
            .str.replace(r"[^\d.\-]", "", regex=True)
        )
        df["amount_paid"] = pd.to_numeric(df["amount_paid"], errors="coerce")
        
        
    ## Step 4: Standardize Categorical Text Fields
    # a) payment status
    status_map = {
        "DONE": "Success", "paid": "Success", "Paid": "Success",
        "success": "Success", "FAILED": "Failed"
    }

    if "payment_status" in df.columns:
        df["payment_status"] = df["payment_status"].astype(str).str.strip().replace(status_map).str.title()
      
     # b) payment channel
    if "payment_channel" in df.columns:
        df["payment_channel"] = df["payment_channel"].astype(str).str.lower().str.strip()
     
    
     # c) discount code cleaning replacing ??? with Nan
    if "discount_code" in df.columns:
        df["discount_code"] = (
            df["discount_code"].astype(str).str.strip()
            .replace({"???": np.nan, "null": np.nan, "": np.nan})
            .apply(lambda x: x.capitalize() if isinstance(x, str) else x)
        )

    # d) invoice number cleaning 
    if "invoice_number" in df.columns:
        df["invoice_number"] = (
            df["invoice_number"].astype(str)
            .str.strip()
            .str.upper()
            .replace({"???": np.nan, "NULL": np.nan, "": np.nan})
            .str.replace(r"[\s_-]*INV[\s_-]*", "Inv-", regex=True) # normalize all INV variations to 'Inv-'
            # Ensure only digits remain after 'Inv-'
            .apply(lambda x: "Inv-" + re.sub(r"[^0-9]", "", x) if isinstance(x, str) and x.startswith("Inv-") else x)
        )

    # e) supoort ticket id cleaning
    if "support_ticket_id" in df.columns:
        df["support_ticket_id"] = (
            df["support_ticket_id"].astype(str)
            .str.strip()
            .replace({"ticket_error": "Tkt-Error"}) # standardize known placeholder 'ticket_error' to 'Tkt-Error'
            .str.replace(r"[\s_-]*TKT[\s_-]*", "Tkt-", regex=True)  # normalize 'TKT', 'tkt', 'Tkt' patterns to 'Tkt-'
             # Ensure numeric part only for IDs that start with 'Tkt-'
            .apply(lambda x: "Tkt-" + re.sub(r"[^0-9]", "", x) if x.startswith("Tkt-") else x)
        )

    
        
     ## Step 5: Clean Refund Flag (Boolean)   
    boolean_map_tx = {"TRUE": True, "Y": True, "YES": True,
                      "FALSE": False, "N": False, "NO": False, "0": False}

    if "refund_flag" in df.columns:
        df["refund_flag"] = (
            df["refund_flag"].astype(str)
            .str.strip().str.upper()
            .map(boolean_map_tx)
            .astype("boolean")   # ensure proper boolean dtype
        )

    duplicates_removed = df.duplicated(subset=["transaction_id"]).sum()
    df.drop_duplicates(subset=["transaction_id"], inplace=True)

    print(f"✅ Transactions cleaned successfully! Removed {duplicates_removed} duplicates.")
    return df

# ================================================
# Apply Cleaning
# ================================================
cleaned_customers = clean_customers(customers)
cleaned_transactions = clean_transactions(transactions)



🧼 Cleaning Customers Data...
✅ Customers cleaned successfully! Removed 15 duplicates.

🧽 Cleaning Transactions Data...
✅ Transactions cleaned successfully! Removed 1429 duplicates.


In [9]:
print("🔍 DATA TYPES AFTER CLEANING")
print("="*50)

print("\n📋 CUSTOMERS TABLE:")
print(customers.dtypes)

print("\n📋 TRANSACTIONS TABLE:")
print(transactions.dtypes)

🔍 DATA TYPES AFTER CLEANING

📋 CUSTOMERS TABLE:
customer_id                  object
full_name                    object
email                        object
country                      object
signup_date          datetime64[ns]
subscription_plan            object
plan_price                    int64
payment_method               object
is_trial                    boolean
renewal_date         datetime64[ns]
churn_flag                  boolean
total_logins                float64
last_login_date      datetime64[ns]
lifetime_value              float64
customer_feedback            object
dtype: object

📋 TRANSACTIONS TABLE:
transaction_id               object
customer_id                  object
transaction_date     datetime64[ns]
amount_paid                   int64
payment_status               object
payment_channel              object
refund_flag                 boolean
invoice_number               object
discount_code                object
support_ticket_id            object
dtype: object


In [10]:
customers.head()

,customer_id,full_name,email,country,signup_date,subscription_plan,plan_price,payment_method,is_trial,renewal_date,churn_flag,total_logins,last_login_date,lifetime_value,customer_feedback
0,1a3d1fa7-bc89-40a9-a3b8-c1e9392456de,Allison Hill,allison@gmail.com,India,NaT,Nan,0,credit card,<NA>,2024-08-21,True,20.0,NaT,260.0,NaN
1,96da1dac-72ff-4d2a-b86e-cbe06b65a6a4,Michelle Miles,michelleoutlook.com,United Kingdom,2000-03-21,Pro,49,Credit Card,False,NaT,True,NaN,2025-06-14,NaN,NaN
2,89463e85-759c-4e66-bacf-b3d00b1f9163,Gregory Baker,gregory@outlook.com,Germany,2021-01-22,Free,0,Credit Card,False,2020-09-01,False,NaN,NaT,NaN,👍 good
3,29a3b2e9-5d65-4441-9588-42dea2bc372f,Brian Ramirez,brian@gmail.com,Germany,NaT,Pro,49,PayPal,<NA>,NaT,False,NaN,2025-04-14,4990.0,Move each left establish.
4,c6a7ee39-c4b0-42cc-97c5-24a55304317f,Melissa Robinson,melissa@outlook.com,Usa,NaT,Nan,49,credit card,<NA>,NaT,False,NaN,NaT,NaN,👍 good


In [11]:
transactions.head()

,transaction_id,customer_id,transaction_date,amount_paid,payment_status,payment_channel,refund_flag,invoice_number,discount_code,support_ticket_id
0,dup-529,61aa189d-0a7d-4de0-bcea-e86bf311a3a1,2024-02-01,19,Failed,credit card,<NA>,Inv-897,Promo10,nan
1,dup-367,8b661d1f-579a-4954-898e-3f5b9134c0b7,2021-11-10,19,0,credit card,False,Inv-950,Newuser,nan
2,NaN,7a70bcf8-92ef-4363-b4de-a4a4d900da4b,2020-03-19,499,Success,credit card,<NA>,NAN,NaN,Tkt-531
3,6d6075e7-a9ce-4f37-ac46-27763d931521,d6c69578-e2e5-4a89-888c-7a64361bc766,2023-03-02,199,0,credit card,False,NaN,Nan,nan
4,ccc40e3c-276b-4087-91ff-622afdf77041,93c16428-c5f9-4dca-b1f0-2b90882cbb08,NaT,499,Success,nan,<NA>,NAN,Newuser,nan


In [12]:
# ================================================
# Validation & Summary Stats
# ================================================

# Foreign key check
invalid_cust = ~cleaned_transactions["customer_id"].isin(cleaned_customers["customer_id"])
invalid_count = invalid_cust.sum()

# ================================================
# Summary Log
# ================================================
summary = {
    "Dataset": ["Customers", "Transactions"],
    "Rows Before": [raw_customers.shape[0], raw_transactions.shape[0]],
    "Rows After": [cleaned_customers.shape[0], cleaned_transactions.shape[0]],
    "Duplicates Removed": [
        raw_customers.duplicated(subset=["customer_id"]).sum(),
        raw_transactions.duplicated(subset=["transaction_id"]).sum()
    ],
    "Invalid Links": [0, invalid_count],
    "Last Cleaned": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")] * 2
}

cleaning_log = pd.DataFrame(summary)
cleaning_log


,Dataset,Rows Before,Rows After,Duplicates Removed,Invalid Links,Last Cleaned
0,Customers,1500,1485,15,0,2025-11-14 13:41:40
1,Transactions,3000,1571,1429,77,2025-11-14 13:41:40


In [16]:
# Replace NaT / NaN / None with empty strings for Sheets compatibility
cleaned_customers = cleaned_customers.astype(object).where(pd.notnull(cleaned_customers), "")
cleaned_transactions = cleaned_transactions.astype(object).where(pd.notnull(cleaned_transactions), "")


In [18]:
# ===================================================
# Save Cleaned Data
# ===================================================

cleaned_customers.to_csv("cleaned_customers.csv", index=False)
cleaned_transactions.to_csv("cleaned_transactions.csv", index=False)
print("💾 Cleaned files saved successfully!")

💾 Cleaned files saved successfully!


In [20]:
#===================================================
# Prepare for Google Sheets
# ===================================================

# Convert all datetime & NaT to clean string format
cleaned_customers = cleaned_customers.astype(str).replace("NaT", "")
cleaned_transactions = cleaned_transactions.astype(str).replace("NaT", "")

#===================================================
# Upload to Google Sheets
# ===================================================

def upload_to_gsheet(sheet_name, dataframe):
    print(f"📤 Uploading {sheet_name}...")

    # Google API scopes
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive"
    ]

    creds = Credentials.from_service_account_file("credentials.json", scopes=scope)
    client = gspread.authorize(creds)

    # Open target Sheet
    sheet = client.open("SaaS Cleaned Data")

    try:
        ws = sheet.worksheet(sheet_name)
        ws.clear()
    except gspread.exceptions.WorksheetNotFound:
        ws = sheet.add_worksheet(title=sheet_name, rows="100", cols="20")

    ws.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())
    print(f"✅ {sheet_name} uploaded successfully!")

# # Upload Data & Log
upload_to_gsheet("Cleaned_Customers", cleaned_customers)
upload_to_gsheet("Cleaned_Transactions", cleaned_transactions)
upload_to_gsheet("Cleaning_Log", cleaning_log)

print("\n✨ All uploads completed successfully!")


📤 Uploading Cleaned_Customers...
✅ Cleaned_Customers uploaded successfully!
📤 Uploading Cleaned_Transactions...
✅ Cleaned_Transactions uploaded successfully!

✨ All uploads completed successfully!


In [21]:
#===================================================
# Upload Data
# ==================================================

upload_to_gsheet("Cleaned_Customers", cleaned_customers)
upload_to_gsheet("Cleaned_Transactions", cleaned_transactions)
upload_to_gsheet("Cleaning_Log", cleaning_log)

print("\n✨ All uploads completed successfully!")


📤 Uploading Cleaned_Customers...
✅ Cleaned_Customers uploaded successfully!
📤 Uploading Cleaned_Transactions...
✅ Cleaned_Transactions uploaded successfully!

✨ All uploads completed successfully!
